In [1]:
import numpy as np
import pandas as pd
import os
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset

from transformers import (
    DataCollatorWithPadding,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)
from datasets import Dataset

from sklearn.preprocessing import MinMaxScaler

os.chdir("/g/data/jr19/rh2942/text-empathy/")
from evaluation import pearsonr
from utils.utils import plot, get_device, set_all_seeds
from utils.common import EarlyStopper

In [2]:
os.environ['TOKENIZERS_PARALLELISM'] = 'false' # due to huggingface warning
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [ ]:
0.607
'''
class RobertaRegressor(nn.Module):
    def __init__(self, checkpoint):
        super(RobertaRegressor, self).__init__()
        
        self.transformer_article = AutoModel.from_pretrained(checkpoint, output_hidden_states=True)
        self.transformer_essay = AutoModel.from_pretrained(checkpoint, output_hidden_states=True)
        
        self.fc1 = nn.Sequential(
            nn.Linear(768*8+1+5, 512), nn.Tanh(), nn.Dropout(0.2)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(512, 256), nn.Tanh(), nn.Dropout(0.2),
            nn.Linear(256, 1)
        )

    def _get_embeddings(self, output):
        layers = [-4, -3, -2, -1] #last four hidden states
        states = output.hidden_states
        embedding = torch.stack([states[i] for i in layers]).sum(0).squeeze()
        # print(embedding.shape)
        return embedding[:, 0] #return only CLS token's embedding
        
    def forward(
        self,
        input_ids_article=None,
        attention_mask_article=None,
        input_ids_essay=None,
        attention_mask_essay=None,
        gender=None,
        education=None,
        race=None,
        age=None,
        income=None
    ):

        output_article = self.transformer_article(
            input_ids= input_ids_article,
            attention_mask=attention_mask_article,
        )

        output_essay = self.transformer_essay(
            input_ids= input_ids_essay,
            attention_mask=attention_mask_essay,
        )

        cosine = nn.functional.cosine_similarity(self._get_embeddings(output_article),
                                                self._get_embeddings(output_essay), dim=1).view(-1, 1)

        output_article = torch.cat([output_article.hidden_states[i] for i in [-4, -3, -2, -1]], dim=-1) #shape: (batch_size, seq_length, 768*4)
        output_essay = torch.cat([output_essay.hidden_states[i] for i in [-4, -3, -2, -1]], dim=-1) #shape: (batch_size, seq_length, 768*4)
        output = torch.cat((output_article[:, 0, :], output_essay[:, 0, :], cosine), dim=1)
        # output = torch.cat((output_article.last_hidden_state[:, 0, :], output_essay.last_hidden_state[:, 0, :], cosine), dim=1)
        output = torch.cat([output, gender, education, race, age, income], 1)

        output = self.fc1(output)
        output = self.fc2(output)
        return output

In [9]:
class RobertaRegressor(nn.Module):
    def __init__(self, checkpoint):
        super(RobertaRegressor, self).__init__()
        
        self.transformer_article = AutoModel.from_pretrained(checkpoint, output_hidden_states=True)
        self.transformer_essay = AutoModelForSequenceClassification.from_pretrained(checkpoint, output_hidden_states=True, num_labels=768)
        
        self.fc1 = nn.Sequential(
            nn.Linear(768, 512), nn.Tanh(), nn.Dropout(0.2)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(512+5+1, 256), nn.Tanh(), nn.Dropout(0.2),
            nn.Linear(256, 1)
        )

    def _get_embeddings(self, output):
        layers = [-4, -3, -2, -1] #last four hidden states
        states = output.hidden_states
        embedding = torch.stack([states[i] for i in layers]).sum(0).squeeze()
        # print(embedding.shape)
        return embedding[:, 0] #return only CLS token's embedding
        
    def forward(
        self,
        input_ids_article=None,
        attention_mask_article=None,
        input_ids_essay=None,
        attention_mask_essay=None,
        gender=None,
        education=None,
        race=None,
        age=None,
        income=None
    ):

        output_article = self.transformer_article(
            input_ids= input_ids_article,
            attention_mask=attention_mask_article,
        )

        output_essay = self.transformer_essay(
            input_ids= input_ids_essay,
            attention_mask=attention_mask_essay,
        )

        cosine = nn.functional.cosine_similarity(self._get_embeddings(output_article),
                                                self._get_embeddings(output_essay), dim=1).view(-1, 1)

        # output_article = torch.cat([output_article.hidden_states[i] for i in [-4, -3, -2, -1]], dim=-1) #shape: (batch_size, seq_length, 768*4)
        # output_essay = torch.cat([output_essay.hidden_states[i] for i in [-4, -3, -2, -1]], dim=-1) #shape: (batch_size, seq_length, 768*4)
        # output = torch.cat((output_article[:, 0, :], output_essay[:, 0, :], cosine), dim=1)
        # output = torch.cat((output_article.pooler_output, output_essay.pooler_output, cosine), dim=1)

        # output = self.fc1(torch.cat((output_essay.logits, cosine), 1))
        output = self.fc1(output_essay.logits)
        output = torch.cat([output, gender, education, race, age, income, cosine], 1)
        output = self.fc2(output)
        return output

In [10]:
class DataModule:
    def __init__(self, task, checkpoint, batch_size, feature_to_tokenise, seed):

        self.task = task
        self.checkpoint = checkpoint
        self.batch_size = batch_size
        self.tokeniser = AutoTokenizer.from_pretrained(
            self.checkpoint,
            use_fast=True
        )
        self.data_collator = DataCollatorWithPadding(tokenizer=self.tokeniser)
        self.feature_to_tokenise = feature_to_tokenise # to tokenise function
        self.seed = seed

        assert len(self.task) == 1, 'task must be a list with one element'
    
    def _process_raw(self, path, send_label):
        data = pd.read_csv(path, sep='\t')
    
        if send_label:
            text = data[self.feature_to_tokenise + self.task]
        else:
            text = data[self.feature_to_tokenise]

        demog = ['gender', 'education', 'race', 'age', 'income']
        data_demog = data[demog]
        scaler = MinMaxScaler()
        data_demog = pd.DataFrame(
            scaler.fit_transform(data_demog),
            columns=demog
        )
        data = pd.concat([text, data_demog], axis=1) 
        return data

    def _tokeniser_fn(self, sentence):
        if len(self.feature_to_tokenise) == 1: # only one feature
            return self.tokeniser(sentence[self.feature_to_tokenise[0]], truncation=True)
        # otherwise tokenise a pair of sentence
        return self.tokeniser(sentence[self.feature_to_tokenise[0]], sentence[self.feature_to_tokenise[1]], truncation=True)

    def _process_input(self, file, send_label):
        data = self._process_raw(path=file, send_label=send_label)
        data = Dataset.from_pandas(data, preserve_index=False) # convert to huggingface dataset
        data = data.map(self._tokeniser_fn, batched=True, remove_columns=self.feature_to_tokenise) # tokenise
        data = data.with_format('torch')
        return data

    # taken from https://pytorch.org/docs/stable/notes/randomness.html
    def _seed_worker(self, worker_id):
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)     

    def dataloader(self, file, send_label, shuffle):
        data = self._process_input(file=file, send_label=send_label)

        # making sure the shuffling is reproducible
        g = torch.Generator()
        g.manual_seed(self.seed)
        
        return DataLoader(
            data,
            batch_size=self.batch_size,
            shuffle=shuffle,
            collate_fn=self.data_collator,
            num_workers=12,
            worker_init_fn=self._seed_worker,
            generator=g
        )

In [11]:
class Trainer:
    def __init__(self, task, model, lr, n_epochs, train_loader_article, train_loader_essay,
                 dev_loader_article, dev_loader_essay, dev_label_file, device_id=0):
        self.device = get_device(device_id)
        self.task = task
        self.model = model.to(self.device)
        self.lr = lr
        self.n_epochs = n_epochs
        self.train_loader_article = train_loader_article
        self.train_loader_essay = train_loader_essay
        self.dev_loader_article = dev_loader_article
        self.dev_loader_essay = dev_loader_essay
        self.dev_label_file = dev_label_file
        
        self.loss_fn = nn.MSELoss()
        self.optimiser = torch.optim.AdamW(
            params=self.model.parameters(),
            lr=self.lr,
            betas=(0.9, 0.98),
            eps=1e-06,
            weight_decay=0.1
        )

        n_training_step = self.n_epochs*len(self.train_loader_article)
        self.lr_scheduler = get_linear_schedule_with_warmup(
            optimizer=self.optimiser,
            num_warmup_steps=0.06*n_training_step,
            num_training_steps=n_training_step
        )
        
        self.best_pearson_r = -1.0 # initiliasation
        self.early_stopper = EarlyStopper(patience=3, min_delta=0.01)
        
        assert len(self.task) == 1, 'task must be a list with one element'

    def _training_step(self):
        tr_loss = 0.0
        self.model.train()
    
        for (data_article, data_essay) in zip(self.train_loader_article, self.train_loader_essay):
            input_ids_article = data_article['input_ids'].to(self.device, dtype=torch.long)
            attention_mask_article = data_article['attention_mask'].to(self.device, dtype=torch.long)
            
            input_ids_essay = data_essay['input_ids'].to(self.device, dtype=torch.long)
            attention_mask_essay = data_essay['attention_mask'].to(self.device, dtype=torch.long)
            
            assert (data_article[self.task[0]].detach().numpy() == data_essay[self.task[0]].detach().numpy()).all(), 'Ground truth is different between encoders'
            targets = data_essay[self.task[0]].to(self.device, dtype=torch.float).view(-1, 1)
            
            gender = data_essay['gender'].to(self.device, dtype=torch.float).view(-1, 1)
            education = data_essay['education'].to(self.device, dtype=torch.float).view(-1, 1)
            race = data_essay['race'].to(self.device, dtype=torch.float).view(-1, 1)
            age = data_essay['age'].to(self.device, dtype=torch.float).view(-1, 1)
            income = data_essay['income'].to(self.device, dtype=torch.float).view(-1, 1)
    
            outputs = self.model(
                input_ids_article=input_ids_article,
                attention_mask_article=attention_mask_article,
                input_ids_essay=input_ids_essay,
                attention_mask_essay=attention_mask_essay,
                gender=gender,
                education=education,
                race=race,
                age=age,
                income=income
            )
            loss = self.loss_fn(outputs, targets)
            tr_loss += loss.item()
    
            self.optimiser.zero_grad()
            loss.backward()
            self.optimiser.step()
            self.lr_scheduler.step()

        epoch_loss = tr_loss / len(train_loader_essay)
        print(f'Train loss: {epoch_loss}')

    def fit(self, save_model=False, gpt_anno=True):
        '''
        gpt_anno: the dev label is differently placed as compared no normal datasets
        '''
        if gpt_anno:
            dev_label = pd.read_csv(self.dev_label_file, sep='\t')
            true = dev_label.loc[:, 'empathy'].tolist()
        else:
            dev_label = pd.read_csv(self.dev_label_file, sep='\t', header=None)
            true = dev_label.iloc[:, 0].tolist()
        
        for epoch in range(self.n_epochs):
            print(f'Epoch: {epoch+1}')
            self._training_step()

            preds = self.evaluate(dataloader_article=self.dev_loader_article,
                                  dataloader_essay=self.dev_loader_essay,
                                  load_model=False)
            
            pearson_r = pearsonr(true, preds)
            print(f'Pearson r: {pearson_r}')
            
            val_loss = self.loss_fn(torch.tensor(preds), torch.tensor(true))
            print('Validation loss:', val_loss.item())
            
            if self.early_stopper.early_stop(val_loss):
                break

            if (pearson_r > self.best_pearson_r):
                self.best_pearson_r = pearson_r            
                if save_model:
                    torch.save(self.model.state_dict(), 'roberta-baseline.pth')
                    print("Saved the model in epoch " + str(epoch+1))
            
            print(f'Best dev set Pearson r: {self.best_pearson_r}\n')

    def evaluate(self, dataloader_article, dataloader_essay, load_model=False):
        if load_model:
            self.model.load_state_dict(torch.load('roberta-baseline.pth'))
    
        pred = torch.empty((len(dataloader_essay.dataset), 1), device=self.device) # len(self.dev_loader.dataset) --> # of samples
        self.model.eval()
    
        with torch.no_grad():
            idx = 0
            for (data_article, data_essay) in zip(dataloader_article, dataloader_essay):
                input_ids_article = data_article['input_ids'].to(self.device, dtype=torch.long)
                attention_mask_article = data_article['attention_mask'].to(self.device, dtype=torch.long)
                
                input_ids_essay = data_essay['input_ids'].to(self.device, dtype=torch.long)
                attention_mask_essay = data_essay['attention_mask'].to(self.device, dtype=torch.long)
                
                assert (data_article['gender'].detach().numpy() == data_essay['gender'].detach().numpy()).all(), 'Sample value is different between encoders'
                
                gender = data_essay['gender'].to(self.device, dtype=torch.float).view(-1, 1)
                education = data_essay['education'].to(self.device, dtype=torch.float).view(-1, 1)
                race = data_essay['race'].to(self.device, dtype=torch.float).view(-1, 1)
                age = data_essay['age'].to(self.device, dtype=torch.float).view(-1, 1)
                income = data_essay['income'].to(self.device, dtype=torch.float).view(-1, 1)
        
                outputs = self.model(
                    input_ids_article=input_ids_article,
                    attention_mask_article=attention_mask_article,
                    input_ids_essay=input_ids_essay,
                    attention_mask_essay=attention_mask_essay,
                    gender=gender,
                    education=education,
                    race=race,
                    age=age,
                    income=income
                )
                
                batch_size = outputs.shape[0]
                pred[idx:idx+batch_size, :] = outputs
                idx += batch_size
            
        return [float(k) for k in pred]   

In [12]:
checkpoint = 'roberta-base'
task = ['empathy']
feature_to_tokenise=['demographic_essay', 'article']
# feature_to_tokenise=['demographic', 'essay']
# feature_to_tokenise=['demographic_essay']
seed = 0

################# COMBINED TRAIN FILE ##############
train_file = './data/WS22-WS23-train-gpt.tsv'
# train_file = './data/essay-train-ws22-ws23.tsv'
# train_file = './data/PREPROCESSED-WS22-WS23-train.tsv'
# train_file = './data/COMBINED-PREPROCESSED-PARAPHRASED-WS22-WS23-train.tsv'

################# WASSA 2022 ####################
# dev_file = './data/PREPROCESSED-WS22-dev.tsv'
# dev_label_file = './data/WASSA22/goldstandard_dev_2022.tsv'
# test_file = './data/PREPROCESSED-WS22-test.tsv'

##### GPT annotation
# train_file = './data/WS22-train-gpt.tsv'
# dev_file = './data/WS22-dev-gpt.tsv'
# dev_label_file = './data/WS22-dev-gpt.tsv'


################# WASSA 2023 ####################
# dev_file = './data/PREPROCESSED-WS23-dev.tsv'
# dev_label_file = './data/WASSA23/goldstandard_dev.tsv'
test_file = './data/PREPROCESSED-WS23-test.tsv'

##### GPT annotation
# train_file = './data/WS23-train-gpt.tsv'
dev_file = './data/WS23-dev-gpt.tsv'
dev_label_file = './data/WS23-dev-gpt.tsv'

In [13]:
set_all_seeds(seed)

data_module_article = DataModule(
    task=task,
    checkpoint=checkpoint,
    batch_size=16,
    feature_to_tokenise=[feature_to_tokenise[1]],
    seed=0
)

data_module_essay = DataModule(
    task=task,
    checkpoint=checkpoint,
    batch_size=16,
    feature_to_tokenise=[feature_to_tokenise[0]],
    seed=0
)

train_loader_article = data_module_article.dataloader(file=train_file, send_label=True, shuffle=True)
train_loader_essay = data_module_essay.dataloader(file=train_file, send_label=True, shuffle=True)
dev_loader_article = data_module_article.dataloader(file=dev_file, send_label=False, shuffle=False)
dev_loader_essay = data_module_essay.dataloader(file=dev_file, send_label=False, shuffle=False)

model = RobertaRegressor(checkpoint=checkpoint)

trainer = Trainer(
    task=task,
    model=model,
    lr=1e-5,
    n_epochs=10,
    train_loader_article=train_loader_article,
    train_loader_essay=train_loader_essay,
    dev_loader_article=dev_loader_article,
    dev_loader_essay=dev_loader_essay,
    dev_label_file=dev_label_file,
    device_id=0
)

trainer.fit(save_model=True, gpt_anno=True)

Map:   0%|          | 0/2634 [00:00<?, ? examples/s]

/scratch/jr19/rh2942/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Map:   0%|          | 0/2634 [00:00<?, ? examples/s]

/scratch/jr19/rh2942/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Map:   0%|          | 0/208 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

/scratch/jr19/rh2942/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bia

Epoch: 1


/scratch/jr19/rh2942/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Train loss: 14.017223392833364
Pearson r: 0.058
Validation loss: 2.689617156982422
Saved the model in epoch 1
Best dev set Pearson r: 0.058

Epoch: 2
Train loss: 3.377618338844993
Pearson r: 0.529
Validation loss: 1.8220162391662598
Saved the model in epoch 2
Best dev set Pearson r: 0.529

Epoch: 3
Train loss: 2.092752405730161
Pearson r: 0.64
Validation loss: 1.2241322994232178
Saved the model in epoch 3
Best dev set Pearson r: 0.64

Epoch: 4
Train loss: 1.4857874155044555
Pearson r: 0.701
Validation loss: 1.1257109642028809
Saved the model in epoch 4
Best dev set Pearson r: 0.701

Epoch: 5
Train loss: 1.2184921373020519
Pearson r: 0.708
Validation loss: 1.0383673906326294
Saved the model in epoch 5
Best dev set Pearson r: 0.708

Epoch: 6
Train loss: 0.9412215841538978
Pearson r: 0.73
Validation loss: 0.9889430403709412
Saved the model in epoch 6
Best dev set Pearson r: 0.73

Epoch: 7
Train loss: 0.7425705148415132
Pearson r: 0.714
Validation loss: 1.0883115530014038
Best dev set Pear

# Test

In [16]:
test_loader_article = data_module_article.dataloader(file=test_file, send_label=False, shuffle=False)
test_loader_essay = data_module_essay.dataloader(file=test_file, send_label=False, shuffle=False)
pred = trainer.evaluate(dataloader_article=test_loader_article,
                        dataloader_essay=test_loader_essay, load_model=True)
pred_df = pd.DataFrame({'emp': pred, 'dis': pred}) # we're not predicting distress, just aligning with submission system
pred_df.to_csv('./tmp/predictions_EMP.tsv', sep='\t', index=None, header=None)
pred_df

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/scratch/jr19/rh2942/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/scratch/jr19/rh2942/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


,emp,dis
0,3.631469,3.631469
1,6.198637,6.198637
2,4.589288,4.589288
3,2.492352,2.492352
4,6.224233,6.224233
...,...,...
95,6.429137,6.429137
96,5.013251,5.013251
97,5.388442,5.388442
98,6.416263,6.416263


# Extra

In [12]:
class RobertaRegressor(nn.Module):
    def __init__(self, checkpoint):
        super(RobertaRegressor, self).__init__()
        
        self.transformer_article = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=768)
        self.transformer_essay = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=768)
        
        self.fc1 = nn.Sequential(
            nn.Linear(768+768+1, 512), nn.Tanh(), nn.Dropout(0.2)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(512+5, 256), nn.Tanh(), nn.Dropout(0.2),
            nn.Linear(256, 1)
        )

    def forward(
        self,
        input_ids_article=None,
        attention_mask_article=None,
        input_ids_essay=None,
        attention_mask_essay=None,
        gender=None,
        education=None,
        race=None,
        age=None,
        income=None
    ):

        output_article = self.transformer_article(
            input_ids= input_ids_article,
            attention_mask=attention_mask_article,
        )

        output_essay = self.transformer_essay(
            input_ids= input_ids_essay,
            attention_mask=attention_mask_essay,
        )

        cosine = nn.functional.cosine_similarity(output_article.logits,
                                                output_essay.logits, dim=1).view(-1, 1)

        output = torch.cat((output_article.logits, output_essay.logits, cosine), dim=1)

        output = self.fc1(output)
        output = torch.cat([output, gender, education, race, age, income], 1)
        output = self.fc2(output)
        return output